In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [5]:
llm.invoke("hi hello how are you?")

AIMessage(content="Hello! I'm just a computer program so I don't have feelings, but I'm here and ready to help. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bl55GvK4jcVGnj3RXyiegEMUPnFjW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8dbf8bcb-942f-4efb-89ea-1f7d627a4fb7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langgraph.types import Command

In [7]:
from langgraph.prebuilt import create_react_agent

In [ ]:
# class AgentState(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], add_messages]
#     Name:str
#     age:int
#     DOB:int

In [8]:
def add_number(state):
    result=state["num1"]+state["num2"]
    print(f"addition is {result}")
    return Command(goto="multiply",update={"sum":result})

In [9]:
state={"num1":10,"num2":20}

In [10]:
add_number(state)

addition is 30


Command(update={'sum': 30}, goto='multiply')

### Creating one dummy multiagent

it is for network/collab multiagent

In [12]:
from langchain_core.tools import tool

In [89]:
@tool
def transfer_to_multiplication_expert():
    """Ask multiplication agent for help"""
    return

In [90]:
@tool
def transfer_to_addition_expert():
    """Ask addition agent for help"""
    return

In [91]:
llm_with_tool=llm.bind_tools([transfer_to_addition_expert])

In [92]:
response=llm_with_tool.invoke("hi")

In [93]:
response.content

'Hello! How can I assist you today?'

In [94]:
response.tool_calls

[]

In [97]:
response=llm_with_tool.invoke("what is 2+2?")

In [98]:
response.content

''

In [99]:
response.tool_calls

[{'name': 'transfer_to_addition_expert',
  'args': {},
  'id': 'call_efUFRfP7dDZYAYVdr6qQ5O0q',
  'type': 'tool_call'}]

In [34]:
system_prompt = (
        "You are an addition expert, you can ask the multiplication expert for help with multiplication."
        "Always do your portion of calculation before the handoff."
    )

In [35]:
messages = [{"role": "system", "content": system_prompt}] + ["can you tell me the addition of 2 and 2?"]

In [36]:
messages

[{'role': 'system',
  'content': 'You are an addition expert, you can ask the multiplication expert for help with multiplication.Always do your portion of calculation before the handoff.'},
 'can you tell me the addition of 2 and 2?']

[{'role': 'system',
  'content': 'You are an addition expert, you can ask the multiplication expert for help with multiplication.Always do your portion of calculation before the handoff.'},
 'can you tell me the addition of 2 and 2?']

In [29]:
from typing_extensions import Literal
from langgraph.graph import MessagesState,StateGraph, START,END

In [101]:
def additional_expert(state:MessagesState)-> Command[Literal["multiplication_expert", "__end__"]]:
    
    system_prompt = (
        "You are an addition expert, you can ask the multiplication expert for help with multiplication."
        "Always do your portion of calculation before the handoff."
    )
    
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    
    ai_msg = llm.bind_tools([transfer_to_multiplication_expert]).invoke(messages)
    
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        
        return Command(
            goto="multiplication_expert", update={"messages": [ai_msg, tool_msg]}
        )
    return {"messages": [ai_msg]}

In [102]:
##Agent2
def multiplication_expert(state:MessagesState)-> Command[Literal["additional_expert", "__end__"]]:
    
    system_prompt = (
        "You are a multiplication expert, you can ask an addition expert for help with addition. "
        "Always do your portion of calculation before the handoff."
    )
    
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    ai_msg = llm.bind_tools([transfer_to_addition_expert]).invoke(messages)
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(goto="additional_expert", update={"messages": [ai_msg, tool_msg]})
    return {"messages": [ai_msg]}

In [103]:
graph=StateGraph(MessagesState)

In [104]:
graph.add_node("additional_expert",additional_expert)
graph.add_node("multiplication_expert",multiplication_expert)

In [105]:
graph.add_edge(START, "additional_expert")

In [106]:
app=graph.compile()

In [107]:
app.invoke({"messages":[("user","what's (3 + 5) * 12. Provide me the output")]})

{'messages': [HumanMessage(content="what's (3 + 5) * 12. Provide me the output", additional_kwargs={}, response_metadata={}, id='504276e8-c619-4691-ac26-295af5c6984d'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HyFaM2WWuNu6xLxTB8bbQf0X', 'function': {'arguments': '{}', 'name': 'transfer_to_multiplication_expert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 86, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bl74ZmahG9pCAdfp8KYhPKfG6oULJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d9308028-8f59-44a9-ba7f-df9bd62647dd-0', tool_calls=[{'name': 'transfer_to_multiplication_expert', 'args': {}

## With realtime tool

In [110]:
from langchain_community.tools import DuckDuckGoSearchRun

In [111]:
search_tool=DuckDuckGoSearchRun()

In [112]:
search_tool.invoke("who is a current pm of uk?")

"British Prime Minister Keir Starmer delivers a speech following his first Cabinet meeting as Prime Minister, in London, July 6, 2024. Mr Starmer has been leader of the Labour party since 2020 and is the country's 58th prime minister. He was born in 1962. His dad was a toolmaker in a factory and his mum was a nurse in the NHS. Keir Starmer, a human rights lawyer from a small town in southern England, is the new prime minister of the United Kingdom. His victory on July 4 was overwhelming. Yet Britain barely knows who he is. Starmer has been a politician for only nine years, and since being crowned U.K. Labour leader in 2020 has steered his party from its worst ... Sir Keir Starmer became Prime Minister on 5 July 2024. Education Keir attended Reigate Grammar School, before studying Law at the University of Leeds. He went on to do postgraduate studies at the ... Exit poll suggests landslide win for Labour Party in U.K. election 01:22. London — When Keir Starmer was elected to lead Britain

In [113]:
from langchain_experimental.utilities import PythonREPL

In [114]:
repl=PythonREPL()

In [115]:
code = """
x = 5
y = x * 2
print(y)
"""

In [117]:
repl.run(code)

Python REPL can execute arbitrary code. Use with caution.


'10\n'

In [118]:
from typing import Annotated

In [119]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )
    

In [120]:
python_repl_tool

StructuredTool(name='python_repl_tool', description='Use this to execute python code. If you want to see the output of a value,\n    you should print it out with `print(...)`. This is visible to the user.', args_schema=<class 'langchain_core.utils.pydantic.python_repl_tool'>, func=<function python_repl_tool at 0x000001BC95CFFAC0>)

In [122]:
print(python_repl_tool.invoke(code))

Successfully executed:
\`\`\`python

x = 5
y = x * 2
print(y)

\`\`\`
Stdout: 10


If you have completed all tasks, respond with FINAL ANSWER.


In [124]:
#agent1
def research_node(state:MessagesState)->Command[Literal["chart_generator", END]]:
    pass

In [125]:
#agent2
def chart_node(state:MessagesState)-> Command[Literal["researcher", END]]:
    pass

In [126]:
workflow = StateGraph(MessagesState)
workflow.add_node("researcher", research_node)
workflow.add_node("chart_generator", chart_node)

workflow.add_edge(START, "researcher")
app = workflow.compile()